In this Jupyter Notebook, we will be creating an interactive Map that includes data from a particular year for accidents recorded by TfL.

This data was provided using the TfL API (https://api.tfl.gov.uk/) - This particular series of cells contain all the information for the earliest recorded year (2005).

In order to download later years, you must sign up for the API and pull the appropriate JSON file.

In [ ]:
import json
# Counts the number of records within a JSON so we can upload the full records to the map
record_count = 0

# Load JSON data
# Replace the 'with open' with the file location of the response.json - This is the JSON that contains the necessary geographic data to map the accidents.
with open('response.json', 'r') as f:
    data = json.load(f)

    # Iterate over the elements of the JSON object
    for element in data:
        try:
            # Increment record counter for each element
            record_count += 1
            
            # Print the keys of the element
            if isinstance(element, dict):
                continue
            else:
                print("Element is not a dictionary.")
        
        except Exception as e:
            print("Error:", e)

# Print total number of records
print("Total number of records:", record_count)

We are using Folium to produce the HTML Map.
I have added additional settings here to ensure we make the most of the data we have downloaded via the API.
I also ensured that the clusters are interactive, to provude a cleaner looking UI and a changing when zooming in.

In [ ]:
import folium
from folium.plugins import MarkerCluster
import json
from collections import Counter
import sys
import time
from IPython.display import clear_output

# Initialize the map
m = folium.Map(location=[51.5074, -0.1278], zoom_start=12)  # London coordinates

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(m)

# Extract latitude, longitude, severity, and vehicle information and save them to a list of dictionaries
incident_data = []

# Counters for casualties, vehicles, and incomplete casualty data
casualty_counter = Counter()
vehicle_counter = Counter()
incomplete_casualty_data = []

# Define a function for printing text with a delay and clearing output 
def print_with_delay(text, clear=True, delay=3):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(0.05)  # Adjust the delay as needed for character-by-character printing
    print()
    
    if delay > 0:
        time.sleep(delay)  # Pause for the specified delay before clearing the output
        
    if clear:
        clear_output(wait=True)  # Clear the previous output before showing the next line

# Replace with file location of response.json
with open('response.json', 'r') as f:
    data = json.load(f)
    if isinstance(data, list):  # Ensure the loaded data is a list
        for record in data:
            _id = record.get('id')
            lat = record.get('lat')
            lon = record.get('lon')
            location = record.get('location')
            date = record.get('date')
            borough = record.get('borough')
            severity = record.get('severity')
            vehicles = record.get('vehicles')
            casualties = record.get('casualties')
            if lat is not None and lon is not None and severity is not None:
                # Flatten the list of vehicles
                vehicles_flat = [vehicle['type'] for vehicle in vehicles] if vehicles else []
                
                # Extract casualty information
                if casualties:
                    for casualty in casualties:
                        age = casualty.get('age')
                        class_ = casualty.get('class')
                        mode = casualty.get('mode')
                        if all([age, class_, mode]):
                            # Increment counters for casualties
                            casualty_counter.update([(age, class_, mode)])
                        else:
                            # Log incomplete casualty data
                            incomplete_casualty_data.append(casualty)
                
                # Increment counter for vehicles
                if vehicles:
                    vehicle_types = [vehicle['type'] for vehicle in vehicles]
                    vehicle_counter.update(vehicle_types)

                incident_data.append({'lat': lat,
                                      'lon': lon,
                                      'severity': severity,
                                      'vehicles': vehicles_flat,
                                      'casualties': casualties,
                                      'id': _id,
                                      'location':location,
                                      'date':date,
                                      'borough':borough})

    else:
        print("The JSON data is not in the expected format.")

# Print number of incidents using the length of incident_data
print_with_delay(f"Number of Incidents: {len(incident_data)}")
print_with_delay("---------------------------------------------Initializing...------------------------------------------------")

# Print severity counts
severity_counts = Counter(incident['severity'] for incident in incident_data)
print_with_delay("SEVERITY: ")
for severity, count in severity_counts.items():
    print_with_delay(f"Count of Severity '{severity}': {count}")
print_with_delay("---------------------------------------------Initializing...------------------------------------------------")

# Print the counts of vehicle types
vehicle_counts = {
    'Car': vehicle_counter['Car'],
    'Taxi': vehicle_counter['Taxi'],
    'Pedal Cycle': vehicle_counter['PedalCycle'],
    'Buses/Coaches': vehicle_counter['BusOrCoach'],
    'Motorcycles (500cc+)': vehicle_counter['Motorcycle_500cc_Plus'],
    'Motorcycles (0-50cc)': vehicle_counter['Motorcycle_0_50cc'],
    'Light Goods Vehicle': vehicle_counter['LightGoodsVehicle'],
    'Medium Goods Vehicle': vehicle_counter['MediumGoodsVehicle'],
    'Heavy Goods Vehicle': vehicle_counter['HeavyGoodsVehicle'],
    'Minibus': vehicle_counter['Minibus'],
    'Other Motor Vehicle(s)': vehicle_counter['OtherMotorVehicle']
}

# Print each line character by character with a delay
for vehicle_type, count in vehicle_counts.items():
    print(f"Vehicle Count: {vehicle_type}: ", end="")
    sys.stdout.flush()  # Flush the output buffer
    for char in str(count):
        print(char, end="", flush=True)
        time.sleep(0.05)  # Adjust the delay as needed
    print()  # Move to the next line after printing the count
    time.sleep(0.5)  # Add additional delay between lines
print_with_delay ("---------------------------------------------Loading Map-------------------------------------------------")

#############################################################################################################
# print(vehicle_counter) - Will need in case there are more Vehicle types in later years.
#############################################################################################################

for incident in incident_data:
    lat = incident['lat']
    lon = incident['lon']
    severity = incident['severity']
    vehicles = incident['vehicles']  # Extract vehicles involved in the incident
    _id = incident.get('id', 'N/A')  # Use 'Unknown ID' if 'id' field is missing
    location = incident.get('location', 'N/A')  # Use 'Unknown Location' if 'location' field is missing
    date = incident.get('date', 'N/A')  # Use 'Unknown Date' if 'date' field is missing
    borough = incident.get('borough', 'N/A')  # Use 'Unknown Borough' if 'borough' field is missing
    
    # Customize marker color based on severity
    if severity == 'Slight':
        color = 'green'
    elif severity == 'Serious':
        color = 'orange'
    elif severity == 'Fatal':
        color = 'red'
    else:
        color = 'blue'  # Default color for unknown severity
          
    # Construct the popup text with severity and vehicle information
    popup_text = f"ID: {_id}<br>"
    popup_text += f"Severity: {severity}<br>"
    popup_text += f"Date: {date}<br>"
    popup_text += f"Location: {location}<br>"
    popup_text += f"Borough: {borough}<br>"
    if vehicles:
        popup_text += f"Vehicles involved:<br>{',  '.join(vehicles)}"
    else:
        popup_text += "No vehicle information available"

    # Create the marker with the customized popup text and add it to the MarkerCluster layer
    folium.Marker([lat, lon], popup=popup_text, icon=folium.Icon(color=color)).add_to(marker_cluster)

# Save the Map as a HTML
print_with_delay ("---------------------------------------------Saving Map--------------------------------------------------")
 m.save('TfL_Incident_Map_2005.html')

# Display the map inline in Jupyter Notebook
m